In [1]:
##################################################################################################
##   Notebook used for extracting text from html files. Some basic preprocessing tasks 
##   v3.0 Unsupervised classification using gensim
##   Required packages: os, logging, collections, gensim
##################################################################################################

In [2]:
## Cross checking the python version that has been installed
##
import platform
platform.python_version()

'2.7.12'

In [3]:
## Importing the logging function which provides logs for debugging
##
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
## Raw corpus used for training n unsupervised gensim model
##
raw_corpus = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",              
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

In [5]:
# Create a set of frequent words. Using only a rudimentary set of stop words
# Kindly note in our use casse removing stop words may create problems
stoplist = set('for a of the and to in'.split(' '))

In [6]:
# Lowercase each document, split it by white space and filter out stopwords
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in raw_corpus]
texts

[['human', 'machine', 'interface', 'lab', 'abc', 'computer', 'applications'],
 ['survey', 'user', 'opinion', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'management', 'system'],
 ['system', 'human', 'system', 'engineering', 'testing', 'eps'],
 ['relation', 'user', 'perceived', 'response', 'time', 'error', 'measurement'],
 ['generation', 'random', 'binary', 'unordered', 'trees'],
 ['intersection', 'graph', 'paths', 'trees'],
 ['graph', 'minors', 'iv', 'widths', 'trees', 'well', 'quasi', 'ordering'],
 ['graph', 'minors', 'survey']]

In [7]:
# Count word frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
frequency

defaultdict(int,
            {'abc': 1,
             'applications': 1,
             'binary': 1,
             'computer': 2,
             'engineering': 1,
             'eps': 2,
             'error': 1,
             'generation': 1,
             'graph': 3,
             'human': 2,
             'interface': 2,
             'intersection': 1,
             'iv': 1,
             'lab': 1,
             'machine': 1,
             'management': 1,
             'measurement': 1,
             'minors': 2,
             'opinion': 1,
             'ordering': 1,
             'paths': 1,
             'perceived': 1,
             'quasi': 1,
             'random': 1,
             'relation': 1,
             'response': 2,
             'survey': 2,
             'system': 4,
             'testing': 1,
             'time': 2,
             'trees': 3,
             'unordered': 1,
             'user': 3,
             'well': 1,
             'widths': 1})

In [8]:
# Only keep words that appear more than once
processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]
processed_corpus

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [9]:
from gensim import corpora

dictionary = corpora.Dictionary(processed_corpus)
print(dictionary)

## Dictionary is stored to disk using save()
dictionary.save('deerwester.dict')

D:\Anaconda2\lib\site-packages\gensim\utils.py:843: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2017-01-23 13:17:47,016 : WARNING : Slow version of gensim.models.doc2vec is being used
2017-01-23 13:17:47,048 : INFO : 'pattern' package not found; tag filters are not available for English
2017-01-23 13:17:47,055 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-01-23 13:17:47,055 : INFO : built Dictionary(12 unique tokens: [u'minors', u'graph', u'system', u'trees', u'eps']...) from 9 documents (total 29 corpus positions)
2017-01-23 13:17:47,056 : INFO : saving Dictionary object under deerwester.dict, separately None
2017-01-23 13:17:47,059 : INFO : saved deerwester.dict


Dictionary(12 unique tokens: [u'minors', u'graph', u'system', u'trees', u'eps']...)


In [10]:
# Simply enumerating through the dictionary
for i in enumerate(dictionary):
    print i

(0, 11)
(1, 10)
(2, 6)
(3, 9)
(4, 8)
(5, 1)
(6, 5)
(7, 7)
(8, 2)
(9, 4)
(10, 0)
(11, 3)


In [11]:
# Better way to do the above
print(dictionary.token2id)

{u'minors': 11, u'graph': 10, u'system': 6, u'trees': 9, u'eps': 8, u'computer': 1, u'survey': 5, u'user': 7, u'human': 2, u'time': 4, u'interface': 0, u'response': 3}


In [12]:
## Check the output with a new document
##
new_doc = 'Human computer interaction'
new_vec = dictionary.doc2bow(new_doc.lower().split())
new_vec

[(1, 1), (2, 1)]

In [13]:
# Converting the entire corpus into a list of vectors
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]
bow_corpus
# Mode persistency by serializing the list of vectors and storing to disk, for later use
corpora.MmCorpus.serialize('deerwester.mm', bow_corpus)  

2017-01-23 13:17:47,448 : INFO : storing corpus in Matrix Market format to deerwester.mm
2017-01-23 13:17:47,450 : INFO : saving sparse matrix to deerwester.mm
2017-01-23 13:17:47,451 : INFO : PROGRESS: saving document #0
2017-01-23 13:17:47,453 : INFO : saved 9x12 matrix, density=25.926% (28/108)
2017-01-23 13:17:47,454 : INFO : saving MmCorpus index to deerwester.mm.index


In [14]:
from gensim import models
# train the model
tfidf = models.TfidfModel(bow_corpus)

2017-01-23 13:17:48,246 : INFO : collecting document frequencies
2017-01-23 13:17:48,246 : INFO : PROGRESS: processing document #0
2017-01-23 13:17:48,247 : INFO : calculating IDF weights for 9 documents and 11 features (28 matrix non-zeros)


In [15]:
#transform the "system minors" sting
print tfidf[dictionary.doc2bow("system minors".lower().split())]

[(6, 0.5898341626740045), (11, 0.8075244024440723)]


In [16]:
## Check what happens when we use 'minor'instead of 'minors'
## this is a word which the model has not seen
print tfidf[dictionary.doc2bow("system minor".lower().split())]

[(6, 1.0)]


In [17]:
## Running the entire corpus through a tfidf transformation
corpus_tfidf = tfidf[bow_corpus]
for doc in corpus_tfidf:
    print(doc)

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(1, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.44424552527467476), (6, 0.3244870206138555), (7, 0.3244870206138555)]
[(0, 0.5710059809418182), (6, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(2, 0.49182558987264147), (6, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (4, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(5, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In [18]:
from gensim import corpora, models, similarities

In [19]:
## model persistency using save & load
## 
dictionary = corpora.Dictionary.load('deerwester.dict')
corpus = corpora.MmCorpus('deerwester.mm')

2017-01-23 13:17:51,161 : INFO : loading Dictionary object from deerwester.dict
2017-01-23 13:17:51,161 : INFO : loaded deerwester.dict
2017-01-23 13:17:51,164 : INFO : loaded corpus index from deerwester.mm.index
2017-01-23 13:17:51,164 : INFO : initializing corpus reader from deerwester.mm
2017-01-23 13:17:51,165 : INFO : accepted corpus with 9 documents, 12 features, 28 non-zero entries


In [20]:
corpus
for c in corpus:
    print(c)

[(0, 1.0), (1, 1.0), (2, 1.0)]
[(1, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 1.0), (7, 1.0)]
[(0, 1.0), (6, 1.0), (7, 1.0), (8, 1.0)]
[(2, 1.0), (6, 2.0), (8, 1.0)]
[(3, 1.0), (4, 1.0), (7, 1.0)]
[(9, 1.0)]
[(9, 1.0), (10, 1.0)]
[(9, 1.0), (10, 1.0), (11, 1.0)]
[(5, 1.0), (10, 1.0), (11, 1.0)]


In [21]:
dictionary.token2id

{u'computer': 1,
 u'eps': 8,
 u'graph': 10,
 u'human': 2,
 u'interface': 0,
 u'minors': 11,
 u'response': 3,
 u'survey': 5,
 u'system': 6,
 u'time': 4,
 u'trees': 9,
 u'user': 7}

In [22]:
## Run the corpus through a LSI (Latent Semantic Indexing) model. Model is run for 2 topics
##
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)

2017-01-23 13:17:52,940 : INFO : using serial LSI version on this node
2017-01-23 13:17:52,940 : INFO : updating model with new documents
2017-01-23 13:17:52,943 : INFO : preparing a new chunk of documents
2017-01-23 13:17:52,944 : INFO : using 100 extra samples and 2 power iterations
2017-01-23 13:17:52,944 : INFO : 1st phase: constructing (12L, 102L) action matrix
2017-01-23 13:17:52,944 : INFO : orthonormalizing (12L, 102L) action matrix
2017-01-23 13:17:52,948 : INFO : 2nd phase: running dense svd on (12L, 9L) matrix
2017-01-23 13:17:52,950 : INFO : computing the final decomposition
2017-01-23 13:17:52,951 : INFO : keeping 2 factors (discarding 43.156% of energy spectrum)
2017-01-23 13:17:52,953 : INFO : processed documents up to #9
2017-01-23 13:17:52,953 : INFO : topic #0(3.341): 0.644*"system" + 0.404*"user" + 0.301*"eps" + 0.265*"response" + 0.265*"time" + 0.240*"computer" + 0.221*"human" + 0.206*"survey" + 0.198*"interface" + 0.036*"graph"
2017-01-23 13:17:52,953 : INFO : topi

In [23]:
## Print out the topics 
##
lsi.print_topics(2)

2017-01-23 13:17:53,548 : INFO : topic #0(3.341): 0.644*"system" + 0.404*"user" + 0.301*"eps" + 0.265*"response" + 0.265*"time" + 0.240*"computer" + 0.221*"human" + 0.206*"survey" + 0.198*"interface" + 0.036*"graph"
2017-01-23 13:17:53,549 : INFO : topic #1(2.542): -0.623*"graph" + -0.490*"trees" + -0.451*"minors" + -0.274*"survey" + 0.167*"system" + 0.141*"eps" + 0.113*"human" + -0.107*"time" + -0.107*"response" + 0.072*"interface"


[(0,
  u'0.644*"system" + 0.404*"user" + 0.301*"eps" + 0.265*"response" + 0.265*"time" + 0.240*"computer" + 0.221*"human" + 0.206*"survey" + 0.198*"interface" + 0.036*"graph"'),
 (1,
  u'-0.623*"graph" + -0.490*"trees" + -0.451*"minors" + -0.274*"survey" + 0.167*"system" + 0.141*"eps" + 0.113*"human" + -0.107*"time" + -0.107*"response" + 0.072*"interface"')]

In [24]:
## Run the entire corpus through the lsi model 
##
corpus_lsi = lsi[corpus_tfidf]
for doc in corpus_lsi:
    print(doc)

[(0, 0.38074310701391734), (1, 0.082050389870936852)]
[(0, 0.7738776922717463), (1, -0.20017851856109192)]
[(0, 0.72176013142126449), (1, 0.16780348860319411)]
[(0, 0.71986853282732621), (1, 0.24534782790425577)]
[(0, 0.51823292135519583), (1, -0.16083692447760997)]
[(0, 0.012746183038294404), (1, -0.4901617924531036)]
[(0, 0.034564816348136761), (1, -0.78697238988812523)]
[(0, 0.046922170256372159), (1, -0.8787747019489105)]
[(0, 0.16590324484677316), (1, -0.74074942773116415)]


In [25]:
## Convert a new document to a bag of words and then run through the lsi model
## 
doc = "Human computer interaction"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow] # convert the query to LSI space
print(vec_lsi)

[(0, 0.46182100453271568), (1, 0.070027665278999535)]


In [26]:
## Again for model persistency store the model to disk
lsi.save('model.lsi') # same for tfidf, lda, ...
lsi = models.LsiModel.load('model.lsi')

2017-01-23 13:17:55,573 : INFO : saving Projection object under model.lsi.projection, separately None
2017-01-23 13:17:55,578 : INFO : saved model.lsi.projection
2017-01-23 13:17:55,579 : INFO : saving LsiModel object under model.lsi, separately None
2017-01-23 13:17:55,581 : INFO : not storing attribute projection
2017-01-23 13:17:55,582 : INFO : not storing attribute dispatcher
2017-01-23 13:17:55,585 : INFO : saved model.lsi
2017-01-23 13:17:55,585 : INFO : loading LsiModel object from model.lsi
2017-01-23 13:17:55,586 : INFO : loading id2word recursively from model.lsi.id2word.* with mmap=None
2017-01-23 13:17:55,588 : INFO : setting ignored attribute projection to None
2017-01-23 13:17:55,588 : INFO : setting ignored attribute dispatcher to None
2017-01-23 13:17:55,589 : INFO : loaded model.lsi
2017-01-23 13:17:55,591 : INFO : loading LsiModel object from model.lsi.projection
2017-01-23 13:17:55,592 : INFO : loaded model.lsi.projection


In [27]:
# transform corpus to LSI space and index it

index = similarities.MatrixSimilarity(lsi[corpus]) 

2017-01-23 13:17:56,118 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2017-01-23 13:17:56,119 : INFO : creating matrix with 9 documents and 2 features


In [28]:
## Save and load 
index.save('deerwester.index')
index = similarities.MatrixSimilarity.load('deerwester.index')

2017-01-23 13:17:56,601 : INFO : saving MatrixSimilarity object under deerwester.index, separately None
2017-01-23 13:17:56,604 : INFO : saved deerwester.index
2017-01-23 13:17:56,605 : INFO : loading MatrixSimilarity object from deerwester.index
2017-01-23 13:17:56,607 : INFO : loaded deerwester.index


In [29]:
sims = index[vec_lsi] # perform a similarity query against the corpus

In [30]:
print(list(enumerate(sims))) # print (document_number, document_similarity) 2-tuples

[(0, 0.99809301), (1, 0.93748635), (2, 0.99844527), (3, 0.9865886), (4, 0.90755945), (5, -0.12416792), (6, -0.10639259), (7, -0.098794639), (8, 0.050041765)]


In [31]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims) # print sorted (document number, similarity score) 2-tuples

[(2, 0.99844527), (0, 0.99809301), (3, 0.9865886), (1, 0.93748635), (4, 0.90755945), (8, 0.050041765), (7, -0.098794639), (6, -0.10639259), (5, -0.12416792)]


In [32]:
raw_corpus = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",              
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

In [33]:
for _, cos in enumerate(sims):
    print cos, raw_corpus[cos[0]]

(2, 0.99844527) The EPS user interface management system
(0, 0.99809301) Human machine interface for lab abc computer applications
(3, 0.9865886) System and human system engineering testing of EPS
(1, 0.93748635) A survey of user opinion of computer system response time
(4, 0.90755945) Relation of user perceived response time to error measurement
(8, 0.050041765) Graph minors A survey
(7, -0.098794639) Graph minors IV Widths of trees and well quasi ordering
(6, -0.10639259) The intersection graph of paths in trees
(5, -0.12416792) The generation of random binary unordered trees
